In [18]:
import numpy as np
import pandas as pd
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Spice.Simulation import Simulation
from PySpice.Spice.BasicElement import SinusoidalVoltageSource

ImportError: cannot import name 'Simulation' from 'PySpice.Spice.Simulation' (/opt/miniconda3/lib/python3.12/site-packages/PySpice/Spice/Simulation.py)

# Simulating the circuit to verify analytic solution

In [17]:
logger = Logging.setup_logging()
circuit = Circuit('Series RL Circuit')
R = 1.0
L = 0.05
V0 = 1.0
omega = 2.0
freq = omega/(2*np.pi) # Computs the frequency of the AC voltage source

# Sets up circuit
circuit.SinusoidalVoltageSource("V", "1", circuit.gnd, amplitude = V0, frequency = freq) # Defines voltage source at point V
circuit.R("1", "1", "2", R) # Adds a resistor between nodes 1 and 2
circuit.L("1", "2", circuit.gnd, L) # Adds an inductor between node 2 and ground

simulation = circuit.simulator(temperature = 25, nominal_temperature = 25)
analysis = simulation.transient(step_time = 0.01, end_time = 0.50)
current = -analysis["I(V)"]
time_values = np.array(analysis.time)
current_array = np.array(current)
analytic_current = current_array[np.argmin(np.abs(time_array - 0.50))]
print(analytic_current)

CDefError: <cdef source string>:7: duplicate declaration of struct ngcomplex

# Euler's Method

In [7]:
def f(t):
    return (10/101)*(np.exp(-20*t) - np.cos(2*t) + 10*np.sin(2*t)) # Defining the solution function

def didt(i, t):
    return 20*np.sin(2*t)-20*i # Defining the differential equation

h = 0.01 # step size

i = 0 # initial condition

timesteps = np.arange(0,0.51,h)

i_values = [] # stores actual value of function at each timestep

for t in timesteps:
    i_values.append(f(t))
    
approximated_values = [] # stores result from Euler's method at each timestep

approximated_values.append(i)

for t in timesteps: # Loop to iterate over all t in the timesteps array
                # Adds each output from Euler's method into the array, starting with the initial condition 
    i = i + h*didt(i, t) # Euler's method
    approximated_values.append(i)

approximated_values.pop() # The counter in the for-loop adds the approximation for t=0.51, this is removed here

local_error = [] # Initialises a list to store the local truncation error 

local_error.append(0)

timesteps_null = np.arange(0.01,0.51,h)

for t in timesteps_null:
    local_error.append(abs(f(t) - f(t-0.01) - h*didt(f(t-0.01), t-0.01))) # Calculates the local truncation error and adds it to the list

steps = [i for i in range(0, 51)]

data = {
    "Steps $(n)$": steps,
    "$t_n$": timesteps,
    "$i(t_n)$": i_values,
    "$i_n$": approximated_values,
    "$epsilon_l$": local_error    
}

df = pd.DataFrame(data) # Converts obtained into a Panda's dataframe
df.to_csv("data.csv", index = False) # Exports dataframe into a csv file